In [6]:
import os
import cv2
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [7]:
TRAIN_PATH = "/content/caltech-transportation_train"
TEST_PATH  = "/content/caltech-transportation_test"


In [8]:
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))

    for idx, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for file in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, file), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
                labels.append(idx)

    return images, np.array(labels), class_names


In [10]:
!unzip /content/caltech-transportation.zip

Archive:  /content/caltech-transportation.zip
   creating: caltech-transportation_test/
   creating: caltech-transportation_test/airplanes/
  inflating: caltech-transportation_test/airplanes/image_0003.jpg  
  inflating: caltech-transportation_test/airplanes/image_0008.jpg  
  inflating: caltech-transportation_test/airplanes/image_0011.jpg  
  inflating: caltech-transportation_test/airplanes/image_0024.jpg  
  inflating: caltech-transportation_test/airplanes/image_0030.jpg  
  inflating: caltech-transportation_test/airplanes/image_0031.jpg  
  inflating: caltech-transportation_test/airplanes/image_0032.jpg  
  inflating: caltech-transportation_test/airplanes/image_0034.jpg  
  inflating: caltech-transportation_test/airplanes/image_0040.jpg  
  inflating: caltech-transportation_test/airplanes/image_0050.jpg  
  inflating: caltech-transportation_test/airplanes/image_0055.jpg  
  inflating: caltech-transportation_test/airplanes/image_0064.jpg  
  inflating: caltech-transportation_test/air

In [11]:
X_train, y_train, class_names = load_images_from_folder(TRAIN_PATH)
X_test, y_test, _ = load_images_from_folder(TEST_PATH)
len(X_train), len(X_test), len(class_names)


(1453, 366, 5)

In [12]:
sift = cv2.SIFT_create()

def extract_sift_features(images):
    descriptors_list = []
    for img in images:
        kp, des = sift.detectAndCompute(img, None)
        descriptors_list.append(des)
    return descriptors_list


In [13]:
train_desc = extract_sift_features(X_train)
test_desc  = extract_sift_features(X_test)


In [14]:
K_STAR = 400


In [15]:
all_descriptors = np.vstack([d for d in train_desc if d is not None])

kmeans = KMeans(n_clusters=K_STAR, random_state=0)
kmeans.fit(all_descriptors)


KMeans(n_clusters=400, random_state=0)

In [16]:
def build_bovw_histograms(descriptors_list, kmeans, K):
    histograms = []
    for descriptors in descriptors_list:
        hist = np.zeros(K)
        if descriptors is not None:
            labels = kmeans.predict(descriptors)
            for label in labels:
                hist[label] += 1
        histograms.append(hist)
    return np.array(histograms)


In [17]:
X_train_bovw = build_bovw_histograms(train_desc, kmeans, K_STAR)
X_test_bovw  = build_bovw_histograms(test_desc, kmeans, K_STAR)

X_train_bovw.shape, X_test_bovw.shape


((1453, 400), (366, 400))

In [18]:
C_values = [0.1, 1, 10]


In [19]:
results = []

for C in C_values:
    print(f"\n▶ Εκτέλεση για C = {C}")

    svm = SVC(kernel="linear", C=C, decision_function_shape="ovr")
    svm.fit(X_train_bovw, y_train)

    preds = svm.predict(X_test_bovw)
    acc = accuracy_score(y_test, preds)

    print(f"   Accuracy = {acc:.4f}")

    results.append({
        "C": C,
        "Accuracy": acc
    })



▶ Εκτέλεση για C = 0.1
   Accuracy = 0.9071

▶ Εκτέλεση για C = 1
   Accuracy = 0.9071

▶ Εκτέλεση για C = 10
   Accuracy = 0.9071


In [20]:
df_results_svm = pd.DataFrame(results)
df_results_svm


,C,Accuracy
0,0.1,0.907104
1,1.0,0.907104
2,10.0,0.907104


In [21]:
df_results_svm.to_csv("experiment_3_svm_C_results.csv", index=False)
print("✅ Αποθήκευση ολοκληρώθηκε: experiment_3_svm_C_results.csv")


✅ Αποθήκευση ολοκληρώθηκε: experiment_3_svm_C_results.csv
